In [2]:
from pyscf import dft, gto
from xtddft.sTDA import os_sTDA
from xtddft.utils import unit, atom

# UsTDA

In [5]:
mol = gto.M(
    atom=atom.ch2o_vacuum,
    # atom=atom.ch2o_cyclohexane,
    unit="A",
    basis='cc-pvdz',
    spin=1,
    charge=1,
    verbose=4
)

# # ground state calculation add solvents
# # add solvents
# mf = dft.UKS(mol).SMD()
# # in https://gaussian.com/scrf/ solvents entry, give different eps for different solvents
# # mf.with_solvent.eps = 2.0165  # for Cyclohexane 环己烷
# # mf.with_solvent.eps = 4.2400  # for DiethylEther 乙醚
# # mf.with_solvent.eps = 7.4257  # for TetraHydroFuran 四氢呋喃

mf = dft.UKS(mol)  # if include solvent effect, remember comment this line
mf.conv_tol = 1e-11
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
xc = 'b3lyp'
mf.xc = xc
# mf.conv_check = False
mf.kernel()

ustda = os_sTDA.OSsTDA(mol, mf)
print("="*50)
ustda.spinadapt = False  # do not use spin adapted
ustda.union = True  # union eigenvector of CV(0) and CV(1) part
ustda.correct = False  # https://doi.org/10.1063/1.4959605
ustda.paramtype = 'os'  # os or cs, os is stand for open shell, cs is for close shell
ustda.nstates = 12
ustda.Emax = 10.0
ustda.cas = True
ustda.info()
e_eV, os, rs, v = ustda.kernel()

System: uname_result(system='Linux', node='localhost', release='6.6.87.2-microsoft-standard-WSL2', version='#1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025', machine='x86_64')  Threads 12
Python 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
numpy 2.4.0  scipy 1.16.3  h5py 3.15.1
Date: Wed Feb 25 10:38:30 2026
PySCF version 2.11.0
PySCF path  /home/whope/miniconda3/envs/stda/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 15
[INPUT] charge = 1
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C      0.000000000000   0.526270000000   0.000000000000 AA    0.000000000000   0.994506167575   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.979180000000   1.091955000000   0.000000000000 AA    1.850382026652   2.06

# XsTDA

In [6]:
mol = gto.M(
    # atom=atom.ch2o_vacuum,
    atom=atom.ch2o_cyclohexane,
    unit="A",
    basis='cc-pvdz',
    spin=1,
    charge=1,
    verbose=4
)

# add solvents
mf = dft.ROKS(mol).SMD()
# in https://gaussian.com/scrf/ solvents entry, give different eps for different solvents
mf.with_solvent.eps = 2.0165  # for Cyclohexane 环己烷
# mf.with_solvent.eps = 4.2400  # for DiethylEther 乙醚
# mf.with_solvent.eps = 7.4257  # for TetraHydroFuran 四氢呋喃

# mf = dft.ROKS(mol)
mf.conv_tol = 1e-11
mf.conv_tol_grad = 1e-8
mf.max_cycle = 200
mf.xc = 'b3lyp'
mf.conv_check = False
mf.kernel()

xstda = os_sTDA.OSsTDA(mol, mf)
print("="*50)
xstda.spinadapt = True  # do not use spin adapted
xstda.union = True  # union eigenvector of CV(0) and CV(1) part
xstda.paramtype = 'os'
xstda.nstates = 12
xstda.cas = True
xstda.Emax = 10
xstda.correct = False
xstda.info()
e_eV, os, rs, v = xstda.kernel()

System: uname_result(system='Linux', node='localhost', release='6.6.87.2-microsoft-standard-WSL2', version='#1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025', machine='x86_64')  Threads 12
Python 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
numpy 2.4.0  scipy 1.16.3  h5py 3.15.1
Date: Wed Feb 25 10:39:07 2026
PySCF version 2.11.0
PySCF path  /home/whope/miniconda3/envs/stda/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 15
[INPUT] charge = 1
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 C     -0.000004000000   0.523281000000   0.000000000000 AA   -0.000007558904   0.988857776189   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.971959000000   1.102186000000   0.000000000000 AA    1.836736314306   2.08